# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 观测结果预处理

---
*@author: Evan*\
*@date: 2023-08-08*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [8]:
xls=pd.read_excel('D:/data/Project_Anqing/202307/obsdata/originaldata.xlsx',header=0,na_values=['—',-99])

xls['datetime'] = pd.to_datetime(xls['日期'].astype(str)+'T'+xls['时间'].astype(str))

xls.drop(['城市','市县','日期','时间'], axis=1, inplace=True)

xls.set_index('datetime',inplace=True)
xls

,站点,SO2,NO2,NOx,NO,CO,O3,PM10,PM2.5,风速,风向,气压,气温,湿度,降水量,能见度
datetime,,,,,,,,,,,,,,,,
2023-07-01 00:00:00,安庆大学,7,16,17,1,0.392,41,20,9,0.4,34.8,999.2,24.9,97,0.000(BB),NaN
2023-07-01 01:00:00,安庆大学,7,16,17,1,0.387,42,14,9,0.6,27.9,999,25,96,0.000(BB),NaN
2023-07-01 02:00:00,安庆大学,7,14,15,1,0.396,36,23,9,0.4,15.6,998.6,24.9,97,0.000(BB),NaN
2023-07-01 03:00:00,安庆大学,7,20,22,2,0.397,31,19,10,0.5,51.1,998.5,24.6,97,0.000(BB),NaN
2023-07-01 04:00:00,安庆大学,8,16,17,1,0.386,32,18,10,0.7,37.2,998.6,24.4,97,0.000(BB),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31 19:00:00,高新区,NaN,8,10,2,0.448,56,27,4,2.3,308.5,998.5,28.7,90,NaN,NaN
2023-07-31 20:00:00,高新区,NaN,10,12,1,0.459,47,19,4,2.3,320.5,999,28.1,93,NaN,NaN
2023-07-31 21:00:00,高新区,NaN,11,14,1,0.524,40,24,6,1.2,291.2,999.6,27.9,95,NaN,NaN


In [9]:
# 将所有因包含括号等字符被读取为字符串的数据设置为空值
xls = xls.replace(to_replace=[r'\(.*?\)'], value=np.nan, regex=True)

In [10]:
xls[xls['站点']=='安庆政务中心']['气压']

datetime
2023-07-01 00:00:00    100.2
2023-07-01 01:00:00    100.1
2023-07-01 02:00:00    100.1
2023-07-01 03:00:00    100.1
2023-07-01 04:00:00    100.1
                       ...  
2023-07-31 19:00:00    100.4
2023-07-31 20:00:00    100.4
2023-07-31 21:00:00    100.5
2023-07-31 22:00:00    100.5
2023-07-31 23:00:00    100.5
Name: 气压, Length: 744, dtype: float64

In [11]:
# 发现安庆政务中心站点的气压数据整体小一个量级
data_replace=xls[xls['站点'].isin(['安庆政务中心'])]['气压']
xls['气压'].replace(data_replace.values,data_replace.values*10,inplace=True)

In [12]:
# 根据站点分组导出
grouped = xls.groupby('站点')
t_index=pd.date_range(timestart,timeend,freq='h')

for group_name, group_df in grouped:
    # 时间序列如有缺失，则补充空值占位
    group_df.reindex(t_index)
    group_df.to_excel(f'D:/Download/{group_name}.xlsx', index=True)

In [14]:
# 将全部站点取平均并导出
xls.groupby(xls.index).mean().to_excel('D:/Download/allsite.xlsx',index=True)

In [13]:
# 将城区站与通道站分开
path = 'D:/data/Project_Anqing/202307/obsdata/'

urban_path=path + 'urban/'
other_path=path + 'others/'
urban_files = [os.path.splitext(filename)[0] for filename in os.listdir(urban_path) if filename.endswith('.xlsx')]
other_files = [os.path.splitext(filename)[0] for filename in os.listdir(other_path) if filename.endswith('.xlsx')]

urban=xls[xls['站点'].isin(urban_files)]
other=xls[xls['站点'].isin(other_files)]

# 平均并导出
urban.groupby(urban.index).mean().to_excel('D:/Download/urban.xlsx',index=True)
other.groupby(other.index).mean().to_excel('D:/Download/others.xlsx',index=True)